# Merge results, clean and remove duplicates
## For Google Play Store (Round 2)

In [1]:
import sys, os, glob, re
import pandas as pd
import numpy as np
from markdownify import markdownify as md

### Define some functions
#### Find a substring between two strings

In [2]:
def find_between(s, first, last):
    try:
        if (last in s):
          start = s.index(first) + len( first)
          end = s.index(last, start)
          return s[start:end]
        else:
          start = s.index(first) + len(first)
          return s[start:]
    except ValueError:
        return ""

### Define some variables

In [3]:
count = 0
fullcount = 0
start_appid = "?id="
end_appid = "&hl="
start_slang = "&hl="
end_slang = "&gl="
file_begin_str = "Google_Search_Results_"
results_path = "scraped/android/data_miner_bystander_02_08_2022/"
clean_dir = "cleaned/android/02_08_2022/"
df = pd.DataFrame()
stats_total_raw = 0
stats_total_cleaned = 0

# create some arrays
domains = ['google.com','google.co.uk','google.de','google.com.au', 'google.com.hk', 'google.co.kr', 'google.co.za', 'google.co.in']
gl = ['us', 'uk', 'de', 'au', 'hk', 'kr', 'za', 'in']
location = ['United States', 'United Kingdom', 'Germany', 'Australia', 'Hong Kong', 'South Korea', 'South Africa', 'India']

#create an empty dataframe to collect some stats
stats = pd.DataFrame(columns = ['Domain', 'Location', 'Raw¹', 'Cleaned', 'Unique²'])


### Iterate through each file and process it
- Read in each CSV file with results collected with Data-Miner Scraper
- Extract appid from URL
- Extract search domain country from filename
- Remove non-app entries based on URL
- Extract store language from URL
- Add new column "pref" (Preference for results in English)
- Add new column "dupe_count" (count of duplicates per search domain
- Save datasets in CSV with and without duplicates
- Create markdown stats file

In [4]:
for file in sorted(glob.iglob(results_path  + '*.csv')):
    
    count = count + 1
    filename = os.path.basename(file)
    
    print()
    print("1) Reading '" + filename + "' ...", end=" ")
    df2 = pd.read_csv(file, sep=',', header = 0, dtype='unicode')
    num_rows = len(df2.index)
    print(str(num_rows) + " rows")
    
    print("2) Extract appid from url ...")
    df2['appid'] = df2.apply(lambda x: find_between(x['URL_clean'], start_appid, end_appid), axis=1)
    df2['appid'].replace(r'\&gl=.*', '', inplace=True, regex=True) #clean appid in case extraction wasnt clean enough
    
    print("3) Extract search domain country from filename ...")
    df2['search_domain'] = filename[len(file_begin_str):len(file_begin_str)+2]
    stats_domain_index = gl.index(filename[len(file_begin_str):len(file_begin_str)+2])
    stats_domain = domains[stats_domain_index]
    stats_location = location[stats_domain_index]
    stats_raw_rows = len(df2.index)
    
    #drop anything other than https://play.google.com/store/apps/details?
    print("4) Remove non-app entries based on url ...", end=" ")
    df2 = df2[df2['URL_clean'].str.contains('details?')]
    num_rows_new = len(df2.index)
    print(str(num_rows - num_rows_new) + " rows deleted.")
     
    print("5) Extract app store language from url (hl) ...")
    df2['store_lang'] = df2.apply(lambda x: find_between(x['URL_clean'], start_slang, end_slang), axis=1)
    df2['store_lang'].replace('', 'en', inplace=True, regex=True) #fill empty with en
    
    #add preference column
    df2['pref'] = np.where(df2['store_lang'].str.contains('en'), True, False)
                   
    #copy dataframe and save cleaned results for specific google domain
    df3 = df2.copy()
    df3.sort_values(by=['appid', 'pref'], inplace=True)
    df3['dupe_count'] = df3.groupby('appid').appid.transform('count')-1
    stats_cleaned_rows = len(df3.index)
    filename_dupes = clean_dir + "dupes_clean_" + re.sub('\d+', str(stats_cleaned_rows), filename)
    df3.to_csv(filename_dupes, index = False)
    print ("6) Created cleaned CSV with duplicates per domain.")
    
    #save a copy of unique records per domain
    df3 = df3.drop_duplicates(subset=['appid'], keep='last')
    stats_unique_rows = len(df3.index)
    filename_uni = clean_dir + "unique_clean_" + re.sub('\d+', str(stats_unique_rows), filename)
    df3.to_csv(filename_uni, index = False)
    print ("7) Created cleaned CSV without duplicates per domain.")
    
    #add dataframe from next csv to existing dataframe
    df = pd.concat([df,df2])
    fullcount = fullcount + num_rows_new
    
    stats.loc[count] = [stats_domain, stats_location, stats_raw_rows, stats_cleaned_rows, stats_unique_rows]
    stats_total_raw = stats_total_raw + stats_raw_rows
    stats_total_cleaned = stats_total_cleaned + stats_cleaned_rows

#count duplicates and add to new column
df.sort_values(by=['appid', 'pref'], inplace=True)
df['dupe_count']=df.groupby('appid').appid.transform('count')-1
    
#save dataframe with dupes to csv
print("------------------------")
df.to_csv(clean_dir + "final_results_bystander_w_dupes.csv", index = False)
print ("1) Created CSV with duplicates " + str(fullcount) + " rows.")

#remove duplicates
df = df.drop_duplicates(subset=['appid'], keep='last')
num_rows_uni = len(df.index)

#save unique dataframe to csv
df.to_csv(clean_dir + "final_results_bystander_unique.csv", index = False)
print ("2) Created CSV without duplicates: " + str(num_rows_uni) + " rows.")

#get stats for unique entries (which entries were kept?)
kept_arr = df['search_domain'].value_counts().reindex(gl, fill_value=0)
kept_arr.sort_index(inplace = True)
stats['Kept³'] = kept_arr.to_numpy()

with open(clean_dir + 'stats.md', 'w') as f:
    stats.loc[count+1] = ["Totals", "", str(stats_total_raw), str(stats_total_cleaned), "–" , str(sum(kept_arr))]
    dfAsString = stats.to_html(index = False)
    h = md(dfAsString, heading_style="ATX")
    f.write(h)
    
print ("3) Saved stats in markdown file")


1) Reading 'Google_Search_Results_au_144.csv' ... 144 rows
2) Extract appid from url ...
3) Extract search domain country from filename ...
4) Remove non-app entries based on url ... 8 rows deleted.
5) Extract app store language from url (hl) ...
6) Created cleaned CSV with duplicates per domain.
7) Created cleaned CSV without duplicates per domain.

1) Reading 'Google_Search_Results_de_199.csv' ... 199 rows
2) Extract appid from url ...
3) Extract search domain country from filename ...
4) Remove non-app entries based on url ... 8 rows deleted.
5) Extract app store language from url (hl) ...
6) Created cleaned CSV with duplicates per domain.
7) Created cleaned CSV without duplicates per domain.

1) Reading 'Google_Search_Results_hk_178.csv' ... 178 rows
2) Extract appid from url ...
3) Extract search domain country from filename ...
4) Remove non-app entries based on url ... 5 rows deleted.
5) Extract app store language from url (hl) ...
6) Created cleaned CSV with duplicates per dom